In [1]:
pip install gym

In [2]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import pandas as pd
import random

In [3]:
from collections import deque
import gym
import numpy as np
import os
import tensorflow as tf

In [4]:
class spamEnv(gym.Env):
  def __init__(self):
    self.data1="/content/drive/MyDrive/csv/dataset_HSP.csv"
    self.data1_x = pd.DataFrame(pd.read_csv(self.data1))
    self.ACTION_LOOKUP = {0: 'not_spam', 1: 'Spam'}

    self.action_space = spaces.Discrete(len(self.ACTION_LOOKUP))
    self.observation_space = spaces.Discrete(self.data1_x.shape[0])

    self.ob = self._get_random_initial_state()
    self.episode_over = False
    self.turns = 0
    self.sum_rewards = 0.0
    self.action = 0.0
    self.current_state_index = 0
    print(self.data1_x.shape[0])
  

    """
                Parameters
                ----------
                action_index :
                Returns
                -------
                ob, reward, episode_over, info : tuple """

  def step(self, predicted_action_index):
    self.turns += 1
    self.predicted_action = self._take_action(predicted_action_index)
    self.reward = self._get_reward(predicted_action_index)
    self.ob = self._get_next_state()
    if self.turns > 10 or self.sum_rewards > 2:
          self.episode_over = True

    return self.ob, self.reward, self.episode_over, {}

  def reset(self):
        """
               Reset the environment and supply a new state for initial state
               :return:
               """

        self.turns = 0
        self.ob = self._get_random_initial_state()
        self.episode_over = False
        self.sum_rewards = 0.0
        return self.ob

  def render(self, mode='human', close=False):
        pass

  def _take_action(self, action_index):
        """
                Take an action correpsonding to action_index in the current state
                :param action_index:
                :return:
                """
        assert action_index < len(self.ACTION_LOOKUP)
        self.action = action_index
        return self.action


  def _get_random_initial_state(self):
    nrand = random.randint(0, self.data1_x.shape[0])
    self.current_state_index = nrand
    return self.data1_x.iloc[nrand]



  def _get_reward(self, predicted_action):
        """
                Get reward for the action taken in the current state
                :return:
                """
        df = self.data1_x
        labelled_action = df.iloc[self.current_state_index]['label']
        reward = 0.0
        if labelled_action == 'S':
            if predicted_action == 1.0:
                reward = 1.0
            else:
                reward = -1.0
        elif labelled_action == 'H':
            if predicted_action == 0.0:
                reward = 1.0
            else:
                reward = -1.0
        self.sum_rewards += reward
        return reward



  def _get_next_state(self):
        """
        Get the next state from current state
        :return:
        """
        df = self.data1_x
        new_state_index = self.current_state_index + 1
        next_state = df.iloc[new_state_index]
        self.current_state_index = new_state_index
        return next_state

  def _seed(self):
        return